# Part 1. Calculation and Analysis of Sentiment Value of Review Text

In [92]:
import json
import os
import jieba
import pandas as pd
from snownlp import SnowNLP
import csv

## 1. Building an sentiment lexicon

### Dictionary for degree adverbs

In [2]:
#Define weights for different degree adverbs
category1 = {
    '绝对': 2,
    '绝对化': 2,
    '绝对性': 2,
    '绝顶': 2,
    '万分': 2,
    '不得了': 2,
    '不可开交': 2,
    '不亦乐乎': 2,
    '不折不扣': 2,
    '彻头彻尾': 2,
    '充分': 2,
    '到头': 2,
    '地地道道': 2,
    '非常': 2,
    '极': 2,
    '极度': 2,
    '极端': 2,
    '极其': 2,
    '极为': 2,
    '截然': 2,
    '尽': 2,
    '惊人地': 2,
    '绝': 2,
    '刻骨': 2,
    '酷': 2,
    '满': 2,
    '满贯': 2,
    '满心': 2,
    '莫大': 2,
    '奇': 2,
    '入骨': 2,
    '甚为': 2,
    '十二分': 2,
    '十分': 2,
    '十足': 2,
    '死': 2,
    '滔天': 2,
    '痛': 2,
    '透': 2,
    '完全': 2,
    '完完全全': 2,
    '万': 2,
    '万般': 2,
    '无比': 2,
    '无度': 2,
    '无可估量': 2,
    '无以复加': 2,
    '无以伦比': 2,
    '要命': 2,
    '要死': 2,
    '已极': 2,
    '已甚': 2,
    '异常': 2,
    '逾常': 2,
    '贼': 2,
    '之极': 2,
    '之至': 2,
    '至极': 2,
    '卓绝': 2,
    '最为': 2,
    '佼佼': 2,
    '郅': 2,
    '綦': 2,
    '齁': 2,
    '最': 2,
}

category2 = {
    '不为过': 1.8,
    '超': 1.8,
    '超额': 1.8,
    '超外差': 1.8,
    '超微结构': 1.8,
    '超物质': 1.8,
    '出头': 1.8,
    '多': 1.8,
    '浮': 1.8,
    '过': 1.8,
    '过度': 1.8,
    '过分': 1.8,
    '过火': 1.8,
    '过劲': 1.8,
    '过了头': 1.8,
    '过猛': 1.8,
    '过热': 1.8,
    '过甚': 1.8,
    '过头': 1.8,
    '过于': 1.8,
    '过逾': 1.8,
    '何止': 1.8,
    '何啻': 1.8,
    '开外': 1.8,
    '苦': 1.8,
    '老': 1.8,
    '偏': 1.8,
    '强': 1.8,
    '溢': 1.8,
    '忒': 1.8
}

category3 = {
    "不过": 1.5,
    "不少": 1.5,
    "不胜": 1.5,
    "惨": 1.5,
    "沉": 1.5,
    "沉沉": 1.5,
    "出奇": 1.5,
    "大为": 1.5,
    "多": 1.5,
    "多多": 1.5,
    "多加": 1.5,
    "多么": 1.5,
    "分外": 1.5,
    "格外": 1.5,
    "够瞧的": 1.5,
    "够戗": 1.5,
    "好不": 1.5,
    "何等": 1.5,
    "很": 1.5,
    "很是": 1.5,
    "坏": 1.5,
    "可": 1.5,
    "老": 1.5,
    "老大": 1.5,
    "良": 1.5,
    "颇": 1.5,
    "颇为": 1.5,
    "甚": 1.5,
    "实在": 1.5,
    "太": 1.5,
    "太甚": 1.5,
    "特": 1.5,
    "特别": 1.5,
    "尤": 1.5,
    "尤其": 1.5,
    "尤为": 1.5,
    "尤以": 1.5,
    "远": 1.5,
    "着实": 1.5,
    "曷": 1.5,
    "碜": 1.5
}

category4 = {
    '大不了': 1.2,
    '多': 1.2,
    '更': 1.2,
    '更加': 1.2,
    '更进一步': 1.2,
    '更为': 1.2,
    '还': 1.2,
    '还要': 1.2,
    '较': 1.2,
    '较比': 1.2,
    '较为': 1.2,
    '进一步': 1.2,
    '那般': 1.2,
    '那么': 1.2,
    '那样': 1.2,
    '强': 1.2,
    '如斯': 1.2,
    '益': 1.2,
    '益发': 1.2,
    '尤甚': 1.2,
    '逾': 1.2,
    '愈': 1.2,
    '愈 ... 愈': 1.2,
    '愈发': 1.2,
    '愈加': 1.2,
    '愈来愈': 1.2,
    '愈益': 1.2,
    '远远': 1.2,
    '越 ... 越': 1.2,
    '越发': 1.2,
    '越加': 1.2,
    '越来越': 1.2,
    '越是': 1.2,
    '这般': 1.2,
    '这样': 1.2,
    '足': 1.2,
    '足足': 1.2,
}

category5 = {
    '点点滴滴': 0.8,
    '多多少少': 0.8,
    '怪': 0.8,
    '好生': 0.8,
    '还': 0.8,
    '或多或少': 0.8,
    '略': 0.8,
    '略加': 0.8,
    '略略': 0.8,
    '略微': 0.8,
    '略为': 0.8,
    '稍': 0.8,
    '稍稍': 0.8,
    '稍微': 0.8,
    '稍为': 0.8,
    '稍许': 0.8,
    '挺': 0.8,
    '未免': 0.8,
    '相当': 0.8,
    '些': 0.8,
    '些微': 0.8,
    '些小': 0.8,
    '一点': 0.8,
    '一点儿': 0.8,
    '一些': 0.8,
    '有点': 0.8,
    '有点儿': 0.8,
    '有些': 0.8
}

category6 = {
    "半点": 0.5,
    "不大": 0.5,
    "不丁点儿": 0.5,
    "不甚": 0.5,
    "不怎么": 0.5,
    "聊": 0.5,
    "没怎么": 0.5,
    "轻度": 0.5,
    "弱": 0.5,
    "丝毫": 0.5,
    "微": 0.5,
    "相对": 0.5
}


In [3]:
# Create a list of 6 dictionaries
dict_list = [category1, category2, category3, category4, category5, category6]

# Merge all dictionaries into a single dictionary
merged_dict = {}
for d in dict_list:
    merged_dict.update(d)

# Save the merged dictionary to a txt file
# Open the file in write mode, encode with utf-8 and write as a json object
with open('sentiment_dic/adv_dic.txt', 'w', encoding='utf-8') as f:
    json.dump(merged_dict, f, ensure_ascii=False)


### Dictionary for negative words 

In [4]:
# Define a function to read a txt file and return its contents as a list of lines
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read().splitlines()

# Read words from negative_dic.txt file
neg_words = read_txt_file('sentiment_dic/base/negative_dic.txt')

# Convert the list of words into a dictionary and set the value of each word as 1
neg_dict = {word: 1 for word in neg_words}

# Save the negative dictionary to a txt file
# Open the file in write mode, encode with utf-8 and write as a json object
with open('sentiment_dic/neg_dic.txt', 'w', encoding='utf-8') as f:
    json.dump(neg_dict, f, ensure_ascii=False)

### Dictionary for emotional words

In [5]:
def read_txt_file(file_path, encoding):
    with open(file_path, 'r', encoding=encoding) as f:
        return f.read().splitlines()

# Define the file paths and encodings for each sentiment word file
SENTIMENT_FILES = [
    ('sentiment_dic/base/ntusd_dic/NTUSD_negative.txt', 'utf-16le'),
    ('sentiment_dic/base/ntusd_dic/NTUSD_positive.txt', 'utf-16le'),
    ('sentiment_dic/base/hownet_dic/hownet_negative_review.txt', 'gbk'),
    ('sentiment_dic/base/hownet_dic/hownet_positive_review.txt', 'gbk'),
    ('sentiment_dic/base/hownet_dic/hownet_negative_sentiment.txt', 'gbk'),
    ('sentiment_dic/base/hownet_dic/hownet_positive_sentiment.txt', 'gbk')
]

# Define the weights for each sentiment word file
SENTIMENT_WEIGHTS = [
    -1, # negative NTUSD
    1,  # positive NTUSD
    -1, # negative HowNet review
    1,  # positive HowNet review
    -2, # negative HowNet sentiment
    2   # positive HowNet sentiment
]

# Create a dictionary of sentiment words and their weights
sentiment_dict = {}
for file_path, encoding in SENTIMENT_FILES:
    words = read_txt_file(file_path, encoding)
    weight = SENTIMENT_WEIGHTS.pop(0)
    for word in words:
        sentiment_dict[word] = weight

# Save the dictionary as a JSON file
with open('sentiment_dic/senti_dic.txt', 'w', encoding='utf-8') as f:
    json.dump(sentiment_dict, f, ensure_ascii=False)

In [6]:
# Formatting the stored dictionaries
with open('sentiment_dic/senti_dic.txt', 'r') as file:
    content = file.read()

content = content.replace(' ":', '":')
with open('sentiment_dic/senti_dic.txt', 'w') as file:
    file.write(content)

In [7]:
# Read the two txt files and convert them into dictionaries
with open('sentiment_dic/senti_dic.txt', 'r', encoding='utf-8') as f:
    dict1 = json.load(f)

with open('sentiment_dic/neg_dic.txt', 'r', encoding='utf-8') as f:
    dict2 = json.load(f)

with open('sentiment_dic/adv_dic.txt', 'r', encoding='utf-8') as f:
    dict3 = json.load(f)

# Remove keys in dict1 that are also in dict2
for key in dict2.keys():
    if key in dict1:
        del dict1[key]
        
# Remove keys in dict1 that are also in dict3
for key in dict3.keys():
    if key in dict1:
        del dict1[key]

# Save the updated dict1 to a new txt file
with open('sentiment_dic/senti_dic_pro.txt', 'w', encoding='utf-8') as f:
    json.dump(dict1, f, ensure_ascii=False)

## 2. Calculation of sentiment value score

### Algorithm design

In [8]:
# Read sentiment dictionary
with open('sentiment_dic/senti_dic_pro.txt', 'r', encoding='utf-8') as f:
    sentiment_dict = json.load(f)

# Read degree adverb dictionary
with open('sentiment_dic/adv_dic.txt', 'r', encoding='utf-8') as f:
    degree_dict = json.load(f)

# Read negation dictionary
with open('sentiment_dic/neg_dic.txt', 'r', encoding='utf-8') as f:
    negation_dict = json.load(f)

# Function to calculate sentiment score for a single sentence
def sentiment_score(sen):
    seg_list = sen.split()  # Convert text to list of words
    emotion_score = 0  # Sentiment score
    neg_count = 0  # Count of negation words
    for i in range(len(seg_list)):
        # Check for sentiment words
        if seg_list[i] in sentiment_dict:
            score = sentiment_dict[seg_list[i]]
            degree_score = 1  # Default degree score is 1
            negation_score = 1  # Default negation score is 1
            # Look for degree and negation words within the valid distance
            for j in range(max(0, i-3), i):
                if seg_list[j] in degree_dict:
                    degree_score *= degree_dict[seg_list[j]]
                elif seg_list[j] in negation_dict:
                    neg_count += 1
            # Determine whether the sentiment score should be reversed based on the number of negation words
            if neg_count % 2 == 0:
                emotion_score += score * degree_score
            else:
                emotion_score -= score * degree_score
            neg_count = 0  # Reset negation count
    return round(emotion_score, 2)

In [9]:
# Testing a typical positive comment
sen = '位置 优秀 从来不 担心 车 没 地方 停 收费 贵 点儿 地下 停车场 设计 很 好 停车 极其 楼 基础设施 完善 逛起来 很 舒服 好评 点赞'
score = sentiment_score(sen)
print(score)

11.0


In [10]:
# Testing a typical negative comment
sen = '座位 没有 想 坐下 休息 地方 没有 逛累 顾客 滚蛋'
score = sentiment_score(sen)
print(score)

-2


In [11]:
# Testing a typical negative comment
sen = '商场 整体 环境 还 行 有点 闷 月份 很 热 吃 饭 热 不行'
score = sentiment_score(sen)
print(score)

1.02


### Apply to previously processed comment text

In [12]:
# Process four text files
for i in range(1, 5):
    # Read in text file
    with open(f'../data/review/processed/reviews_{i}.txt', 'r', encoding='utf-8') as f:
        reviews = f.readlines()

    # Calculate sentiment score and polarity for each review
    sentiment_scores = []
    polarity = []
    for review in reviews:
        score = sentiment_score(review)
        sentiment_scores.append(score)
        if score < 0:
            polarity.append('N') # If sentiment score is negative, assign 'N' for negative polarity
        elif score >= 1:
            polarity.append('P') # If sentiment score is positive, assign 'P' for positive polarity
        else:
            polarity.append('I') # If sentiment score is neutral, assign 'I' for neutral polarity

    # Create a DataFrame for the reviews and their sentiment scores and polarity
    df = pd.DataFrame(reviews, columns=['review_text'])
    df['sentiment_score'] = sentiment_scores
    df['polarity'] = polarity

    # Save the DataFrame as a CSV file
    df.to_csv(f'../data/review/processed/with_sentiment_score/reviews_{i}.csv', index=False)


## 3. Use snownlp to help verify comment polarity

In [13]:
# Process four text files
for i in range(1, 5):
    # Read the CSV file
    df = pd.read_csv(f'../data/review/processed/with_sentiment_score/reviews_{i}.csv')

    # Read the review text file
    with open(f'../data/review/reviews_{i}.txt', 'r', encoding='utf-8') as f:
        reviews = f.readlines()

    # Calculate sentiment score and polarity for each review and store in lists
    sentiment_scores = []
    polarities = []
    for review in reviews:
        s = SnowNLP(review)
        sentiment_scores.append(s.sentiments)
        if s.sentiments < 0.4:
            polarities.append('N')
        elif s.sentiments < 0.6:
            polarities.append('I')
        else:
            polarities.append('P')

    # Add the sentiment scores and polarities as new columns to the DataFrame
    df['snownlp'] = sentiment_scores
    df['polarity_snow_nlp'] = polarities

    # Write the modified data back to the original CSV file
    df.to_csv(f'../data/review/processed/with_sentiment_score/reviews_{i}.csv', index=False)

## 4. Further data analysis

### Data Concatenation and Organization

In [93]:
for i in range(1, 5):
    # Read in the first CSV file
    df_csv_1 = pd.read_csv(f'../data/review/processed/with_sentiment_score/reviews_{i}.csv')
    # Read in the second CSV file
    df_csv_2 = pd.read_csv(f'../data/review/reviews_{i}.csv')
    # Concatenate the two DataFrames horizontally
    df_concat = pd.concat([df_csv_1, df_csv_2], axis=1)
    # Save the concatenated DataFrame as a CSV file
    df_concat.to_csv(f'../data/review/analysis/reviews_{i}.csv', index=False)

In [94]:
# Create an empty dictionary to store DataFrames
dfs = {}

# Loop through CSV files
for i in range(1, 5):
    # Read in a CSV file
    df = pd.read_csv(f'../data/review/analysis/reviews_{i}.csv')
    
    # Get the name of the last column
    last_col = df.columns[-1]
    
    # Move the last column to the beginning of the DataFrame
    df = df.iloc[:, [df.shape[1]-1] + list(range(df.shape[1]-1))]
    
    # Rename the columns for consistency
    df = df.rename(columns={
        last_col: 'review',
        'review_text': 'review_splitting',
        'sentiment_score': 'senti_score',
        'polarity': 'polarity',
        'snownlp': 'senti_score_snownlp',
        'polarity_snow_nlp': 'polarity_snownlp'
    })
    
    # Add the DataFrame to the dictionary
    dfs[f'reviews_{i}'] = df

In [95]:
# Remove newline characters from the "review" and "review_splitting" columns
for key in dfs.keys():
    # Get the current DataFrame
    review = dfs[key]
    
    # Select the columns to remove newline characters from
    columns_to_clean = ['review', 'review_splitting']
    
    # Use str.replace() method to remove newline characters
    for col in columns_to_clean:
        review[col] = review[col].str.replace('\n', '')
        
    # Save the modified DataFrame back to the original CSV file
    review.to_csv(f'../data/review/analysis/{key}.csv', index=False)

In [96]:
# Assign DataFrames to variables
reviews_1_df = dfs['reviews_1']
reviews_2_df = dfs['reviews_2']
reviews_3_df = dfs['reviews_3']
reviews_4_df = dfs['reviews_4']

### Consistency Check for Polarity Labeling of Review Texts

In [97]:
def compare_columns(df, col1, col2):
    """
    Compares two columns of a pandas DataFrame row by row to check if they are equal, and calculates the ratio 
    of rows that are not equal.

    Args:
        df: pandas DataFrame, the data to compare.
        col1: str, the name of the first column to compare.
        col2: str, the name of the second column to compare.

    Returns:
        A percentage value of the number of rows that are not equal to the total number of rows in the DataFrame.
    """
    # Create an empty list to store the row numbers where the two columns differ.
    diff_rows = []
    
    # Iterate over each row of the DataFrame and compare the values of the two specified columns.
    for i in range(len(df)):
        if df.loc[i, col1] != df.loc[i, col2]:
            # If the values are different, append the row number to the list of differing rows.
            diff_rows.append(i)

    # Calculate the percentage of rows that are different between the two columns.
    diff_ratio = len(diff_rows) / len(df) * 100

    # Print the results of the comparison.
    if len(diff_rows) == 0:
        print(f"{col1} 和 {col2} 两列数据完全一致")
    else:
        print(f"{col1} 和 {col2} 两列数据共有 {len(diff_rows)} 行不一致，占全部行数的 {diff_ratio:.2f}%，即两种算法于该数据集表现出的一致性为 {100 - diff_ratio:.2f}%")

    # Return the percentage of differing rows.
    return diff_ratio

In [98]:
# Calculate consistency ratio for four dataframes by comparing two columns using 'compare_columns' function.
diff_ratio_1 = compare_columns(reviews_1_df, 'polarity', 'polarity_snownlp')
diff_ratio_2 = compare_columns(reviews_2_df, 'polarity', 'polarity_snownlp')
diff_ratio_3 = compare_columns(reviews_3_df, 'polarity', 'polarity_snownlp')
diff_ratio_4 = compare_columns(reviews_4_df, 'polarity', 'polarity_snownlp')

polarity 和 polarity_snownlp 两列数据共有 1165 行不一致，占全部行数的 27.36%，即两种算法于该数据集表现出的一致性为 72.64%
polarity 和 polarity_snownlp 两列数据共有 691 行不一致，占全部行数的 21.79%，即两种算法于该数据集表现出的一致性为 78.21%
polarity 和 polarity_snownlp 两列数据共有 512 行不一致，占全部行数的 28.65%，即两种算法于该数据集表现出的一致性为 71.35%
polarity 和 polarity_snownlp 两列数据共有 343 行不一致，占全部行数的 27.05%，即两种算法于该数据集表现出的一致性为 72.95%


In [99]:
# Calculate the average difference ratio by adding the four ratios and dividing by four.
diff_ratios = [diff_ratio_1, diff_ratio_2, diff_ratio_3, diff_ratio_4]
avg_ratio = sum(diff_ratios) / len(diff_ratios)

# Print the average difference ratio as a percentage with two decimal places.
print(f"Overall consistency: {100 - avg_ratio:.2f}%")

Overall consistency: 73.79%


### Calculate the distribution of review text polarity (in percentages) for the four stores

In [100]:
# This function calculates the percentage of each unique value in a given column of a DataFrame
def calculate_percentages(df, column_name):
    # Count the number of occurrences of each unique value in the specified column
    counts = df[column_name].value_counts()

    # Calculate the percentage of each unique value in relation to the total number of values in the column
    percentages = counts / counts.sum() * 100

    # Return the percentages
    return percentages

In [101]:
# Call the calculate_percentages function for each DataFrame to get the percentage of each unique value in the "polarity" column
percentages_1 = calculate_percentages(reviews_1_df, "polarity")
percentages_2 = calculate_percentages(reviews_2_df, "polarity")
percentages_3 = calculate_percentages(reviews_3_df, "polarity")
percentages_4 = calculate_percentages(reviews_4_df, "polarity")

# Print the results
print("北京APM：")
print(percentages_1)

print("王府中环：")
print(percentages_2)

print("王府井百货：")
print(percentages_3)

print("东方新天地：")
print(percentages_4)


北京APM：
P    75.598873
N    13.785815
I    10.615312
Name: polarity, dtype: float64
王府中环：
P    82.749921
N    10.028382
I     7.221697
Name: polarity, dtype: float64
王府井百货：
P    73.363179
N    15.836598
I    10.800224
Name: polarity, dtype: float64
东方新天地：
P    73.974763
N    14.826498
I    11.198738
Name: polarity, dtype: float64


In [102]:
reviews_1_df["polarity"].value_counts()

P    3219
N     587
I     452
Name: polarity, dtype: int64

### Calculate the Mean value of sentiment_score for the four stores

In [103]:
# Loop through each DataFrame in the dictionary and calculate the average of the 'senti_score' column
for name, df in dfs.items():
    senti_score_mean = df['senti_score'].mean()
    print(f"Mean of 'senti_score' column for {name}: {senti_score_mean:.2f}")

Mean of 'senti_score' column for reviews_1: 4.27
Mean of 'senti_score' column for reviews_2: 5.00
Mean of 'senti_score' column for reviews_3: 3.22
Mean of 'senti_score' column for reviews_4: 3.33


## 5. Store comment text in categories based on polarity

In [104]:
# for key in dfs.keys():
#     # Get the current DataFrame
#     review = dfs[key]
    
#     # Group the DataFrame by the "polarity" column
#     groups = review.groupby('polarity')
    
#     # Extract DataFrames for "P" and "N" polarities from the groups
#     review_p = groups.get_group('P')
#     review_n = groups.get_group('N')
    
#     # Select only the "review" column and write it to a txt file
#     review_p['review'].to_csv(f'../data/review/analysis/polarity/{key}_polarity_P.txt', index=False, header=False)
#     review_n['review'].to_csv(f'../data/review/analysis/polarity/{key}_polarity_N.txt', index=False, header=False)

In [105]:
for key in dfs.keys():
    # Get the current DataFrame
    review = dfs[key]
    
    # Group the DataFrame by the "polarity" column
    groups = review.groupby('polarity')
    
    # Extract DataFrames for "P" and "N" polarities from the groups
    review_p = groups.get_group('P')
    review_n = groups.get_group('N')
    
    # Select only the "review splitting" column and write it to a txt file
    review_p['review_splitting'].to_csv(f'../data/review/analysis/polarity/{key}_polarity_P.txt', index=False, header=False)
    review_n['review_splitting'].to_csv(f'../data/review/analysis/polarity/{key}_polarity_N.txt', index=False, header=False)

## 6. Splitting text into words

In [90]:
# # Set the file path for the stop words
# stopword_path = 'stopwords/stopwords.txt'

# # Create a set of stop words by opening the file and using a set comprehension
# with open(stopword_path, 'r', encoding='utf-8') as f:
#     stopwords = {word.strip() for word in f}

# # Load the custom dictionary
# custom_dict_path = 'stopwords/custom_dict.txt'
# jieba.load_userdict(custom_dict_path)

# # Define a function to process each review text
# def process_text(text):
#     # Tokenize the text with Jieba and enable HMM
#     tokens = jieba.cut(text, cut_all=False, HMM=True)
#     # Remove stop words and punctuation
#     tokens = [token for token in tokens if token not in stopwords and token.strip() and not token.isnumeric()]
#     # Return the processed text as a string
#     return ' '.join(tokens)

In [91]:
# # Set the directory path to the review files
# dir_path = '../data/review/analysis/polarity'

# # Loop through each file in the directory
# for filename in os.listdir(dir_path):
#     # Check if the file is a txt file
#     if filename.endswith('.txt'):
#         # Construct the file path
#         file_path = os.path.join(dir_path, filename)
#         # Read in the contents of the file
#         with open(file_path, 'r', encoding='utf-8') as f:
#             text = f.read()
#         # Process the text using the process_text function
#         processed_text = process_text(text)
#         # Write the processed text back to the file
#         with open(file_path, 'w', encoding='utf-8') as f:
#             f.write(processed_text)